?

In [ ]:
!pip install gTTS  

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pathlib
import pickle
import glob
from tqdm import tqdm
from skimage import io 
from collections import Counter
from PIL import Image
from IPython import display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Model,activations
from keras.utils import plot_model
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from gtts import gTTS

In [ ]:
#Import the dataset and read the image into a seperate variable
data = '/kaggle/input/flickr8k/'
images= os.path.join(data,'Images/')

all_imgs = glob.glob(images + '*.jpg',recursive=True)
print("The total images present in the dataset: {}".format(len(all_imgs)))
all_imgs[:2]

In [ ]:
df_caption = pd.read_csv("/kaggle/input/flickr8k/captions.txt")
df_caption.head()
len(df_caption)

In [ ]:
#Visualise both the images & text present in the dataset
def show_image_and_its_caption(index):
    image_name, caption = df_caption.iloc[index]["image"], df_caption.iloc[index]["caption"]
    img = io.imread(pathlib.Path(images,image_name))
    plt.imshow(img)
    print(caption)

In [ ]:
show_image_and_its_caption(4)

In [ ]:
#Import the dataset and read the text file into a seperate variable
text_file = 'captions.txt'
def load_doc(filename):
    
    text = pd.read_csv(data + filename)
    
    return text

doc = load_doc(text_file)
print(doc[:300])

In [ ]:
all_img_id = df_caption['image']
all_img_vector = [data + 'Images/' + str(img) for img in all_img_id]
annotations = df_caption['caption']

In [ ]:
data = {'ID': all_img_id,'Path':all_img_vector,'Captions':annotations}
df = pd.DataFrame(data=data)     
df

In [ ]:
#Create a list which contains all the captions
annotations= ['<start> '+ text +' <end>' for text in df.Captions]
#add the <start> & <end> token to all those captions as well

#Create a list which contains all the path to the images
all_img_path= list(df.Path)

print("Total captions present in the dataset: "+ str(len(annotations)))
print("Total images present in the dataset: " + str(len(all_img_path)))
print (annotations[:2])

In [ ]:
#Create the vocabulary & the counter for the captions 
concat_str = ' '.join([str(text).strip('\n') for text in annotations]) 
#print(concat_str[:500])
vocabulary= concat_str.split(' ')
print(vocabulary[:50])
print("vocabulary length: "+ str(len(set(vocabulary))))
val_count=Counter(vocabulary)
#val_count


In [ ]:
#Visualise the top 30 occuring words in the captions
wordcloud = WordCloud(
                          background_color='white',
                          max_words=30,
                          max_font_size=40,
                          random_state=42
                         ).generate(str(annotations))

plt.imshow(wordcloud)
plt.axis('off')
plt.show();

In [ ]:
#Visualise the top 30 occuring words in the captions
def plot_top_words(word_counter):
    plt.style.use('fivethirtyeight')
    plt.figure(figsize=(14,4))
    plt.xticks(rotation=45)
    plt.bar(*zip(*word_counter))
    plt.title("Top 30 Occuring Words")
    plt.show()

plot_top_words(val_count.most_common(30))


In [ ]:
# create the tokenizer
top_word_cnt = 5000
tokenizer = Tokenizer(num_words = top_word_cnt +1, filters= '!"#$%^&*()_+.,:;-?/~`{}[]|\=@ ',
                      lower = True, char_level = False, 
                      oov_token = 'UNK')

In [ ]:
tokenizer.fit_on_texts(annotations)
#transform each text into a sequence of integers
text_seqs = tokenizer.texts_to_sequences(annotations)

In [ ]:
text_seqs[:4]

In [ ]:
word_index = tokenizer.word_index
index_word = tokenizer.index_word

tokenizer.word_index['PAD'] = 0
tokenizer.index_word[0] = 'PAD'

In [ ]:
print(tokenizer.oov_token)
print(tokenizer.index_word[0])

In [ ]:
word_count_dict = dict(tokenizer.word_counts.items())
#word_count_dict
len(word_count_dict)
len(tokenizer.index_word)

In [ ]:
## Save Tokenizer
with open('./tokenizer.pkl',"wb") as f:
    pickle.dump(tokenizer,f)

In [ ]:
# Create a word count of your tokenizer to visulize the Top 30 occuring words after text processing
sort_word_by_count = sorted(tokenizer.word_counts.items(), key=lambda kv : kv[1], reverse= True)  #kv[1]: word frequency, reverse = True : desc order
plot_top_words(sort_word_by_count[:30])

In [ ]:
wordcloud = WordCloud(background_color='black',max_words=30,
                          max_font_size=40,
                          random_state=42).generate_from_frequencies(word_count_dict)
plt.imshow(wordcloud)
plt.axis('off')
plt.show();

In [ ]:
# Pad each vector to the max_length of the captions  store it to a vairable

cap_vector= tf.keras.preprocessing.sequence.pad_sequences(text_seqs, padding= 'post', 
                                                          dtype='int32', value=0) 
print("The shape of Caption vector is :" + str(cap_vector.shape))

In [ ]:
IMAGE_SHAPE = (299,299)

In [ ]:
#write your code here for creating the function. This function should return images & their path

def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SHAPE)
    img = preprocess_input(img)
    return img, image_path

# Check the preprocessing Logic
print(f'Size of image before preprocessing: {io.imread(all_img_vector[0]).shape}')
print(f'Size of image after preprocessing: {preprocess_image(all_img_vector[0])[0].shape}')
    
    

In [ ]:
len(sorted(set(all_img_vector)))

In [ ]:
#write your code here to create the dataset consisting of image paths
#write your code here for applying the function to the image path dataset, such that the transformed dataset should contain images & their path

unique_img_paths = sorted(set(all_img_vector))
img_data = tf.data.Dataset.from_tensor_slices(unique_img_paths)
print(img_data)
img_data = img_data.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)
                
img_data

In [ ]:
## Load the pretrained Imagenet weights of Inception net V3

1.To save the memory(RAM) from getting exhausted, extract the features of the images using the last layer of pre-trained model. Including this as part of training will lead to higher computational time.

2.The shape of the output of this layer is 8x8x2048. 

3.Use a function to extract the features of each image in the train & test dataset such that the shape of each image should be (batch_size, 8*8, 2048)



image_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')

new_input = image_model.input  #write code here to get the input of the image_model
hidden_layer = image_model.layers[-1].output  #write code here to get the output of the image_model

image_features_extract_model = tf.keras.Model(new_input, hidden_layer) #build the final model using both input & output layer

In [ ]:
# write the code to apply the feature_extraction model to your earlier created dataset which contained images & their respective paths
# Once the features are created, you need to reshape them such that feature shape is in order of (batch_size, 8*8, 2048)
fv_dict = {}
for img,path in tqdm(img_data):
    fv_out = image_features_extract_model(img)
    fv_flattened = tf.reshape(fv_out,(fv_out.shape[0],-1,fv_out.shape[3])) 
    for fv_img,path in zip(fv_flattened,path):
        path = path.numpy().decode("utf-8") #This decoding is done because the values are not in ASCII
        fv_dict[path] = fv_img.numpy()  #This will convert the tensor instance to numpy

In [ ]:
print(fv_out.shape) # Last batch has only 27 images (126*64 + 27 = 8091)
print(fv_out[0].shape)
print(fv_flattened.shape)
print(fv_flattened[0].shape)

In [ ]:
len(fv_dict)

In [ ]:
#view top five items of features dict
import more_itertools
top_5 = more_itertools.take(5, fv_dict.items())
top_5

## Dataset creation
1.Apply train_test_split on both image path & captions to create the train & test list. Create the train-test spliit using 80-20 ratio & random state = 42

2.Create a function which maps the image path to their feature. 

3.Create a builder function to create train & test dataset & apply the function created earlier to transform the dataset

2.Make sure you have done Shuffle and batch while building the dataset

3.The shape of each image in the dataset after building should be (batch_size, 8*8, 2048)

4.The shape of each caption in the dataset after building should be(batch_size, max_len)


In [ ]:
#write your code here

path_train, path_test, cap_train, cap_test = train_test_split(all_img_vector, cap_vector, test_size = 0.2, random_state = 42)

In [ ]:
print("Training data for images: " + str(len(path_train)))
print("Testing data for images: " + str(len(path_test)))
print("Training data for Captions: " + str(len(cap_train)))
print("Testing data for Captions: " + str(len(cap_test)))

In [ ]:
# Create a function which maps the image path to their feature. 
# This function will take the image_path & caption and return it's feature & respective caption.

def map_func(img_path, cap):
    img_tensor = fv_dict[img_path.decode('utf-8')]
    return img_tensor, cap

### FAQs on how to load the features:
* You can load the features using the dictionary created earlier OR
* You can store using numpy(np.load) to load the feature vector.

In [ ]:
# create a builder function to create dataset which takes in the image path & captions as input
# This function should transform the created dataset(img_path,cap) to (features,cap) using the map_func created earlier
# tf.numpy_function() : Wraps a python function and uses it as a TensorFlow op.

BUFFER_SIZE = 1000
BATCH_SIZE = 64
def gen_dataset(img_path, cap):
        
    dataset = tf.data.Dataset.from_tensor_slices((img_path, cap))
    dataset = dataset.map(lambda ele1, ele2 : tf.numpy_function(map_func, [ele1, ele2], [tf.float32, tf.int32]),num_parallel_calls = tf.data.experimental.AUTOTUNE)           
    dataset = (dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration= True).batch(BATCH_SIZE , drop_remainder = False).prefetch(tf.data.experimental.AUTOTUNE))
    return dataset



In [ ]:
train_dataset=gen_dataset(path_train,cap_train)
test_dataset=gen_dataset(path_test,cap_test)

In [ ]:
sample_img_batch, sample_cap_batch = next(iter(train_dataset))
print(sample_img_batch.shape)  #(batch_size, 8*8, 2048)
print(sample_cap_batch.shape) #(batch_size,max_len)

## Model Building
1.Set the parameters

2.Build the Encoder, Attention model & Decoder

In [ ]:
embedding_dim = 256 
units = 512
vocab_size = 5001 #top 5,000 words +1
train_num_steps = len(path_train)  // BATCH_SIZE
test_num_steps = len(path_test) // BATCH_SIZE

In [ ]:
train_num_steps

In [ ]:
test_num_steps

Encoder

In [ ]:
class Encoder(Model):
    def __init__(self,embed_dim):
        super(Encoder, self).__init__()
        self.dense = layers.Dense(embed_dim) #build your Dense layer with relu activation
        
    def call(self, features):
        features =  self.dense(features)# extract the features from the image shape: (batch, 8*8, embed_dim)
        features = activations.relu(features)
        return features
    
    def model(self):
        features = keras.Input(shape = (64,2048), batch_size = 64)     
        
        return Model(inputs = features, outputs = self.call(features))

In [ ]:
encoder=Encoder(embedding_dim)

In [ ]:
plot_model(encoder.model(),to_file = 'encoder.png', show_shapes = True)

In [ ]:
encoder.model().summary()

## Attention Model

In [ ]:
class Attention_model(Model):
    def __init__(self, units):
        super(Attention_model, self).__init__()
        self.W1 =  layers.Dense(units)  #build your Dense layer for image feature vector
        self.W2 =  layers.Dense(units)   #build your Dense layer for hidden state
        self.V =   layers.Dense(1)  #build your final Dense layer with unit 1 for flattening after addition of fv and hidden state
        self.units = units

    def call(self, features, hidden):
        #features shape: (batch_size, 8*8, embedding_dim). This is the output of Encoder.
        # hidden shape: (batch_size, hidden_size)
        hidden_with_time_axis =  tf.expand_dims(hidden, axis = 1)   # Expand the hidden shape to shape: (batch_size, 1, hidden_size)
        score = self.V(activations.tanh(self.W1(features) + self.W2(hidden_with_time_axis)))  # build your score funciton to shape: (batch_size, 8*8, 1)
        attention_weights = activations.softmax(score,axis = 1)   # extract your attention weights with shape: (batch_size, 8*8, 1)
        context_vector = attention_weights * features   #shape: create the context vector with shape (batch_size, 8*8,embedding_dim)
        context_vector = tf.reduce_sum(context_vector, axis=1)  # reduce the shape to (batch_size, embedding_dim)
        
        return context_vector, attention_weights
    
    def model(self):
        features = keras.Input(shape = (64,256), batch_size = 64 ) 
        hidden = keras.Input(shape = (512), batch_size = 64)
        
        return Model(inputs = [features,hidden], outputs = self.call(features,hidden))

In [ ]:
attention = Attention_model(units)

In [ ]:
attention.model().summary()

In [ ]:
plot_model(attention.model(), to_file = 'attention.png',show_shapes = True)

Decoder

In [ ]:
class Decoder(Model):
    def __init__(self, embed_dim, units, vocab_size):
        super(Decoder, self).__init__()
        self.units = units
        self.attention = Attention_model(self.units) #iniitalise your Attention model with units
        self.embed =  layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)  #build your Embedding layer
        self.gru = tf.keras.layers.GRU(self.units,return_sequences=True,return_state=True,recurrent_initializer='glorot_uniform')
        self.d1 = layers.Dense(self.units)  #build your Dense layer
        self.d2 = layers.Dense(vocab_size)  #build your Dense layer
        

    def call(self,x,features, hidden):
        context_vector, attention_weights = self.attention(features, hidden)  #create your context vector & attention weights from attention model
        embed =  self.embed(x)  # embed your input to shape: (batch_size, 1, embedding_dim)
        embed =  tf.concat([tf.expand_dims(context_vector, 1), embed], axis=-1) # Concatenate your input with the context vector from attention layer. Shape: (batch_size, 1, embedding_dim + embedding_dim)
        output,state = self.gru(embed)  # Extract the output & hidden state from GRU layer. Output shape : (batch_size, max_length, hidden_size)
        output = self.d1(output)
        output = tf.reshape(output, (-1, output.shape[2])) # shape : (batch_size * max_length, hidden_size)
        output = self.d2(output) # shape : (batch_size * max_length, vocab_size)
        
        return output,state, attention_weights
    
    def init_state(self, batch_size):
        return tf.zeros((batch_size, self.units))  # Initial hidden state (shape = 512)
    
    def model(self):
        x = keras.Input(shape = (1), batch_size = 64)  # shape=1 refers to the word_index of the word at that timestamp
        features = keras.Input(shape = (64,256), batch_size = 64 ) 
        hidden = keras.Input(shape = (512), batch_size = 64)
        
        return Model(inputs = [x,features,hidden], outputs = self.call(x,features,hidden))
    
    

In [ ]:
decoder=Decoder(embedding_dim, units, vocab_size)

In [ ]:
decoder.model().summary()

In [ ]:
plot_model(decoder.model(),to_file = 'decoder.png' ,show_shapes = True,show_layer_activations=True,
    show_trainable=True, expand_nested=True)

In [ ]:
#This section verifies the models build, using a sample batch

features=encoder(sample_img_batch)
hidden = decoder.init_state(batch_size=sample_cap_batch.shape[0])
dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * sample_cap_batch.shape[0], 1)  # Providing 64 <start> tokens to indicate start of 64 captions

predictions, hidden_out, attention_weights= decoder(dec_input, features, hidden)

print('Feature shape from Encoder: {}'.format(features.shape)) #(batch, 8*8, embed_dim)
print('Shape of hidden: {}'.format(hidden.shape)) 
print('Shape of dec_input: {}'.format(dec_input.shape))
print('Predictions shape from Decoder: {}'.format(predictions.shape)) #(batch,vocab_size)
print('Shape of hidden_out from Decoder: {}'.format(hidden_out.shape))
print('Attention weights shape from Decoder: {}'.format(attention_weights.shape)) #(batch, 8*8, 1)

In [ ]:
## Model training & optimization
1.Set the optimizer & loss object

2.Create your checkpoint path

3.Create your training & testing step functions

4.Create your loss function for the test dataset

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001)  
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')  #Since decoder does not have a softmax therefore, logits = True

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
!ls /kaggle/working

In [ ]:
if not os.path.exists("/kaggle/working/checkpoints/"):
    os.mkdir("/kaggle/working/checkpoints/") 

In [ ]:
checkpoint_path = "/kaggle/working/checkpoints/"
#checkpoint_path = os.path.join("checkpoints","train")
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)



In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

[link text](https://)* While creating the training step for your model, you will apply Teacher forcing.
* Teacher forcing is a technique where the target/real word is passed as the next input to the decoder instead of previous prediciton.

In [ ]:
@tf.function
def train_step(img_tensor, target):   
    loss = 0
    hidden = decoder.init_state(batch_size=target.shape[0])
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    
    with tf.GradientTape() as tape:
        features = encoder(img_tensor)
        for i in range(1,target.shape[1]):
            predictions,hidden,_ = decoder(dec_input, features, hidden)
            loss += loss_function(target[:, i], predictions)  
            dec_input = tf.expand_dims(target[:, i], 1)  #Teacher enforcing
        avg_loss = (loss/target.shape[1])  #Avg loss over a caption
        trainable_variables = encoder.trainable_variables + decoder.trainable_variables
        gradients = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(gradients, trainable_variables))
           
    return loss, avg_loss

In [ ]:
@tf.function
def test_step(img_tensor, target):
    loss = 0
    hidden = decoder.init_state(batch_size=target.shape[0])
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    features = encoder(img_tensor)
    for i in range(1,target.shape[1]):
        predictions,hidden,_ = decoder(dec_input, features, hidden)
        loss += loss_function(target[:, i], predictions) 
        predicted_id = tf.argmax(predictions,1)  # No Teacher enforcing
        dec_input = tf.expand_dims(predicted_id, 1)
         
    avg_loss = (loss/target.shape[1])  #Avg loss over a batch   
        
    return loss, avg_loss

In [ ]:
def test_loss_cal(test_dataset):
    total_loss = 0
    for (batch, (img_tensor, target)) in enumerate(test_dataset):
        batch_loss, t_loss = test_step(img_tensor, target)
        total_loss += t_loss
    avg_test_loss=total_loss/test_num_steps
    
    return avg_test_loss

In [ ]:
loss_plot = []
test_loss_plot = []
EPOCHS = 15

best_test_loss=100
for epoch in tqdm(range(0, EPOCHS)):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(train_dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss
        avg_train_loss=total_loss / train_num_steps
        
    loss_plot.append(avg_train_loss)    
    test_loss = test_loss_cal(test_dataset)
    test_loss_plot.append(test_loss)
    
    print ('For epoch: {}, the train loss is {:.3f}, & test loss is {:.3f}'.format(epoch+1,avg_train_loss,test_loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
    if test_loss < best_test_loss:
        print('Test loss has been reduced from %.3f to %.3f' % (best_test_loss, test_loss))
        best_test_loss = test_loss
                
        ckpt_manager.save()
      

In [ ]:
plt.plot(loss_plot)
plt.plot(test_loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

In [ ]:
!ls
!cd ./checkpoints/
!ls checkpoints/

In [ ]:
!cat ./checkpoints/checkpoint

In [ ]:
print('shpe of loss_plot', loss_plot)
print('shpe of test_loss_plot', test_loss_plot)

## Model Evaluation
1.Define your evaluation function using greedy search

2.Define your evaluation function using beam search ( optional)

3.Test it on a sample data using BLEU score 

### Greedy Search

In [ ]:
attention_features_shape = 64
max_seq_length = cap_vector.shape[1]
def evaluate(image):
    attention_plot = np.zeros((max_seq_length, attention_features_shape))

    hidden = decoder.init_state(batch_size=1)

    temp_input = tf.expand_dims(preprocess_image(image)[0], 0) #process the input image to desired format before extracting features
    img_tensor_val = image_features_extract_model(temp_input)  # Extract features using our feature extraction model
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)  # extract the features by passing the input to encoder
    #print("shape of features", features.shape)
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    #print("shape of first dec_input", dec_input.shape)
    result = []

    for i in range(max_seq_length):
        predictions, hidden, attention_weights = decoder(dec_input,
                                                         features,
                                                         hidden)# get the output from decoder
        #print("shape of predictions", predictions.shape)
        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()  
        predicted_id = (tf.argmax(predictions,1)).numpy()[0]  #extract the predicted id(index value from tokenizer) which carries the max value
        #print("predicted_id:", predicted_id)
        #map the id to the word from tokenizer and append the value to the result list
        result.append(tokenizer.index_word[predicted_id])
        
        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot,predictions

        dec_input = tf.expand_dims([predicted_id], 0)
        
        
    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot,predictions

'''
Shape of features: (1,64,256)
Shape of first dec_input: (1,1)
Shape of predictions: (1,5001)
'''



In [ ]:
attention_features_shape = 64
max_seq_length = cap_vector.shape[1]
def beam_evaluate(image, beam_index = 3):

    start = [tokenizer.word_index['<start>']]
    result = [[start, 0.0]]

    attention_plot = np.zeros((max_seq_length, attention_features_shape))

    hidden = decoder.init_state(batch_size=1)

    temp_input = tf.expand_dims(preprocess_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

    while len(result[0][0]) < max_seq_length:
        temp = []
        for i, s in enumerate(result):
            predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
            attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
            word_preds = np.argsort(predictions[0])[-beam_index:]

            for w in word_preds:
                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)

                prob += np.log(predictions[0][w])

                temp.append([next_cap, prob])
        result = temp
        result = sorted(result, reverse=False, key=lambda l: l[1])
        result = result[-beam_index:]


        predicted_id = result[-1]
        pred_list = predicted_id[0]

        prd_id = pred_list[-1]
        if(prd_id!=3):
            dec_input = tf.expand_dims([prd_id], 0)  
        else:
            break


    result2 = result[-1][0]

    intermediate_caption = [tokenizer.index_word[i] for i in result2]
    final_caption = []
    for i in intermediate_caption:
        if i != '<end>':
            final_caption.append(i)

        else:
            break

    attention_plot = attention_plot[:len(result), :]
    final_caption = ' '.join(final_caption[1:])
    return final_caption

In [ ]:
test_image = "/kaggle/input/flickr8k/Images/3106883334_419f3fb16f.jpg"
captions=beam_evaluate(test_image)
print(captions)
Image.open(test_image)

In [ ]:
def plot_attmap(caption, weights, image):

    fig = plt.figure(figsize=(10, 10))
    temp_img = np.array(Image.open(image))
    
    len_cap = len(caption)
    for cap in range(len_cap):
        weights_img = np.reshape(weights[cap], (8,8))
        weights_img = np.array(Image.fromarray(weights_img).resize((224, 224), Image.LANCZOS))
        
        ax = fig.add_subplot(len_cap//2, len_cap//2, cap+1)
        ax.set_title(caption[cap], fontsize=15)
        
        img=ax.imshow(temp_img)
        
        ax.imshow(weights_img, cmap='gist_heat', alpha=0.6,extent=img.get_extent())
        ax.axis('off')
    plt.subplots_adjust(hspace=0.2, wspace=0.2)
    plt.show()

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
def filt_text(text):
    filt=['<start>','UNK','<end>'] 
    temp= text.split()
    [temp.remove(j) for k in filt for j in temp if k==j]
    text=' '.join(temp)
    return text

In [ ]:
def test_caption_generation(img_test_batch):
    rid = np.random.randint(0, len(img_test_batch))
    test_image = img_test_batch[rid]
#test_image = './images/413231421_43833a11f5.jpg'
#real_caption = '<start> black dog is digging in the snow <end>'

    real_caption = ' '.join([tokenizer.index_word[i] for i in cap_test[rid] if i not in [0]])
    result, attention_plot,pred_test = evaluate(test_image)


    real_caption=filt_text(real_caption)      


    pred_caption=' '.join(result).rsplit(' ', 1)[0]
      
    real_appn = []
    real_appn.append(real_caption.split())
    reference = real_appn
    candidate = pred_caption.split()

    score = sentence_bleu(reference, candidate, weights= (0.5,0.5,0,0))  #set your weights
    print(f"BELU score: {score*100}")

    print ('Real Caption:', real_caption)
    print ('Prediction Caption:', pred_caption)
    plot_attmap(result, attention_plot, test_image)
    
    Image.open(test_image)
    
    ##Converting text to speech
    tts = gTTS(pred_caption, lang = 'en', slow = False)
    #This can be downloaded and played
    filename = '/kaggle/working/caption.mp3'
    tts.save(filename) 
    display.display(display.Audio(filename, rate = None, autoplay = False)) # To display playback bar as output
    

In [ ]:
test_caption_generation(path_test)